In [87]:
from enum import IntEnum
class IntEnumShortStr(IntEnum):
  def __str__(self):
    return self.name

  def __format__(self, fmt):
    return self.__str__()


EACH_GROUP_SIZE=5
TOTAL_GROUPS_SIZE=EACH_GROUP_SIZE*3
class Difficulty(IntEnumShortStr):
  Easy = 1
  Med = 2
  Hard = 3

# For Easy, many values are just to keep the algorithm happy
DiffThresh = {Difficulty.Easy:100, Difficulty.Med:75, Difficulty.Hard:60}
DiffLimits = {Difficulty.Easy:100, Difficulty.Med:4, Difficulty.Hard:2}
DiffGain = {Difficulty.Easy:0, Difficulty.Med:3, Difficulty.Hard:2}

from random import randint
def generateNextBatch(prev_difficulties_dict, cur_perf_dict):
  def createDiff(diff, range_lim_lo, range_lim_hi):
    new_diff = prev_difficulties_dict[diff] + \
               (1 if cur_perf_dict[diff] < DiffThresh[diff] else -1)*DiffGain[diff]
    lo_lim, hi_lim = new_diff - DiffLimits[diff], new_diff + DiffLimits[diff]
    lo_lim, hi_lim = max(lo_lim, range_lim_lo, 0), min(hi_lim, range_lim_hi, 100)
    if lo_lim > hi_lim:
        hi_lim = min(lo_lim+1, 100)
    new_diff = min(max(range_lim_lo+1, new_diff), range_lim_hi-1)
    #print(f"Difficulty: {diff} - old_diff: {prev_difficulties_dict[diff]} - new_diff: {new_diff} - "
    #      f"Low lim: {lo_lim} - Hi lim: {hi_lim}")
    #print("Gain:",(1 if cur_perf_dict[diff] < DiffThresh[diff] else -1)*DiffGain[diff])
    new_diff_arr = []
    for i in range(EACH_GROUP_SIZE):
      new_diff_arr.append(randint(lo_lim, hi_lim))
    return new_diff, new_diff_arr, (lo_lim, hi_lim)
  # Start with medium difficulty
  med_diff, med_diff_arr, (med_lo, med_hi) = createDiff(Difficulty.Med, 8, 50)
  easy_diff, easy_diff_arr, _ = createDiff(Difficulty.Easy, med_hi + 5, 101)
  hard_diff, hard_diff_arr, _ = createDiff(Difficulty.Hard, 0, med_lo-1)
  new_difficulties_dict = {
    Difficulty.Easy:easy_diff,
    Difficulty.Med:med_diff,
    Difficulty.Hard:hard_diff,
  }
  difficulties_arr_dict = {
    Difficulty.Easy:easy_diff_arr,
    Difficulty.Med:med_diff_arr,
    Difficulty.Hard:hard_diff_arr,
  }
  return new_difficulties_dict, difficulties_arr_dict

def test():
  prev_difficulties_dict = {Difficulty.Easy:100, Difficulty.Med:15, Difficulty.Hard:5}
  cur_perf_dict = DiffThresh.copy()
  from numpy.random import normal
  for i in range(10):
    print(f"\nCurrent perf:: {' - '.join([f'{diff}: {perf}%' for diff, perf in cur_perf_dict.items()])}")
    new_difficulties_dict, difficulties_arr_dict = generateNextBatch(prev_difficulties_dict, cur_perf_dict)
    print(f"New Difficulties: {' - '.join([f'{diff}: {cohr}% cohr' for diff, cohr in new_difficulties_dict.items()])}")
    _ = "%"
    print(f"Difficulties arr: {' - '.join([f'{diff}: {[(str(cohr)+_) for cohr in arr]}' for diff, arr in difficulties_arr_dict.items()])}")
    prev_difficulties_dict = new_difficulties_dict
    cur_perf_dict = {Difficulty.Easy:min(100, DiffThresh[Difficulty.Easy] + normal(0,5)),
                     Difficulty.Med:DiffThresh[Difficulty.Med] + normal(0,5),
                     Difficulty.Hard:max(40, DiffThresh[Difficulty.Hard] + normal(0,5))}

test()


Current perf:: Easy: 100% - Med: 75% - Hard: 60%
New Difficulties: Easy: 100% cohr - Med: 12% cohr - Hard: 3% cohr
Difficulties arr: Easy: ['38%', '24%', '21%', '32%', '31%'] - Med: ['8%', '14%', '15%', '11%', '11%'] - Hard: ['5%', '4%', '1%', '4%', '5%']

Current perf:: Easy: 100% - Med: 65.6509492096124% - Hard: 56.143930639280754%
New Difficulties: Easy: 100% cohr - Med: 15% cohr - Hard: 5% cohr
Difficulties arr: Easy: ['62%', '41%', '70%', '53%', '74%'] - Med: ['13%', '19%', '12%', '15%', '17%'] - Hard: ['3%', '5%', '3%', '4%', '4%']

Current perf:: Easy: 99.88510861194864% - Med: 65.38108790817907% - Hard: 56.636562052739166%
New Difficulties: Easy: 100% cohr - Med: 18% cohr - Hard: 7% cohr
Difficulties arr: Easy: ['58%', '51%', '90%', '43%', '91%'] - Med: ['20%', '20%', '14%', '15%', '20%'] - Hard: ['9%', '6%', '7%', '8%', '5%']

Current perf:: Easy: 100% - Med: 78.30887213022672% - Hard: 55.838695720142745%
New Difficulties: Easy: 100% cohr - Med: 15% cohr - Hard: 9% cohr
Diffi

In [88]:
def evaluateBatchPerf(trials):
  choices_by_diff = {Difficulty.Easy:[], Difficulty.Med:[], Difficulty.Hard:[]}
  for trial in trials:
    choices_by_diff[trial.Difficulty].append(1 if trial.Correct else 0)
  perf = {}
  for diff in Difficulty:
    perf[diff] = 100.0*sum(choices_by_diff[diff])/len(choices_by_diff[diff])
  return perf

def test():
  prev_difficulties_dict = {Difficulty.Easy:100, Difficulty.Med:15, Difficulty.Hard:5}
  import numpy as np
  from numpy.random import rand
  print(rand(5) + DiffThresh[Difficulty.Easy])
  class Trial:
    def __init__(self, diff, correct):
      self.Difficulty = diff
      self.Correct = correct
  for i in range(10):
    theshs = [(Difficulty.Easy, 0.95), (Difficulty.Med, 0.75), (Difficulty.Hard, 0.6)]
    trials = [Trial(diff, is_correct) for diff, thresh in theshs for is_correct in (rand(EACH_GROUP_SIZE) < thresh)]
    #print(f"\nCurrent perf:: {' - '.join([f'{trial.Difficulty}: {trial.Correct}' for trial in trials])}")
    batch_perf = evaluateBatchPerf(trials)

    ###
    print(f"Evaluated Perf: {' - '.join([f'{diff}: {perf}% cohr' for diff, perf in batch_perf.items()])}")
    new_difficulties_dict, difficulties_arr_dict = generateNextBatch(prev_difficulties_dict, batch_perf)
    print(f"New Difficulties: {' - '.join([f'{diff}: {cohr}% cohr' for diff, cohr in new_difficulties_dict.items()])}")
    _ = "%"
    print(f"Difficulties arr: {' - '.join([f'{diff}: {[(str(cohr)+_) for cohr in arr]}' for diff, arr in difficulties_arr_dict.items()])}")
    prev_difficulties_dict = new_difficulties_dict
test()

[100.35580255 100.23884514 100.27082571 100.20334126 100.62927945]
Evaluated Perf: Easy: 100.0% cohr - Med: 60.0% cohr - Hard: 80.0% cohr
New Difficulties: Easy: 100% cohr - Med: 18% cohr - Hard: 3% cohr
Difficulties arr: Easy: ['88%', '35%', '89%', '34%', '62%'] - Med: ['21%', '19%', '20%', '17%', '14%'] - Hard: ['3%', '1%', '2%', '4%', '3%']
Evaluated Perf: Easy: 100.0% cohr - Med: 100.0% cohr - Hard: 60.0% cohr
New Difficulties: Easy: 100% cohr - Med: 15% cohr - Hard: 1% cohr
Difficulties arr: Easy: ['27%', '43%', '99%', '71%', '43%'] - Med: ['15%', '14%', '19%', '13%', '19%'] - Hard: ['3%', '3%', '0%', '2%', '1%']
Evaluated Perf: Easy: 100.0% cohr - Med: 40.0% cohr - Hard: 40.0% cohr
New Difficulties: Easy: 100% cohr - Med: 18% cohr - Hard: 3% cohr
Difficulties arr: Easy: ['85%', '34%', '91%', '77%', '32%'] - Med: ['20%', '14%', '18%', '18%', '15%'] - Hard: ['1%', '5%', '2%', '4%', '3%']
Evaluated Perf: Easy: 100.0% cohr - Med: 40.0% cohr - Hard: 40.0% cohr
New Difficulties: Easy: 